In [16]:
!pip install -Uq openai-agents

In [17]:
import nest_asyncio
nest_asyncio.apply()

In [18]:
from pydantic import BaseModel
from agents import (
    AsyncOpenAI,
    OpenAIChatCompletionsModel,
    RunConfig)
from google.colab import userdata

In [32]:
gemini_api_key = userdata.get("GEMINI_API_KEY")


# Check if the API key is present; if not, raise an error
if not gemini_api_key:
    raise ValueError("GEMINI_API_KEY is not set. Please ensure it is defined in your .env file.")

#Reference: https://ai.google.dev/gemini-api/docs/openai
external_client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

model = OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client=external_client
)

config = RunConfig(
    model=model,
    model_provider=external_client,
    tracing_disabled=True
)

In [33]:
# from agents import set_default_openai_client, set_tracing_disabled
# set_default_openai_client(external_client)
# set_tracing_disabled(True)

In [34]:
from agents import Agent , Runner
import asyncio
# Agent specializing in billing inquiries
billing_agent = Agent(
    name="Billing Agent",
    instructions="You handle all billing-related inquiries. Provide clear and concise information regarding billing issues."
)

# Agent specializing in refund processes
refund_agent = Agent(
    name="Refund Agent",
    instructions="You handle all refund-related processes. Assist users in processing refunds efficiently."
)

# Triage agent that decides which specialist agent to hand off tasks to
triage_agent = Agent(
    name="Triage Agent",
    instructions="You determine which agent should handle the user's request based on the nature of the inquiry.",
    handoffs=[billing_agent, refund_agent]
)

async def main(input: str):
    result = await Runner.run(triage_agent, input=input, run_config=config)
    print(result.final_output)


In [36]:
asyncio.run(main("tell me about my billing status"))

Okay, I am transferring you to a billing agent who can provide you with detailed information about your billing status. Please wait a moment while I connect you.



### Customizing handoffs via the handoff() function

In [37]:
from agents import Agent, Runner, RunContextWrapper, handoff

urdu_agent = Agent(
    name="Urdu agent",
    instructions="You only speak Urdu."
)

english_agent = Agent(
    name="English agent",
    instructions="You only speak English"
)
def on_handoff(agent: Agent, ctx: RunContextWrapper[None]):
    agent_name = agent.name
    print("--------------------------------")
    print(f"Handing off to {agent_name}...")
    print("--------------------------------")

triage_agent = Agent(
    name="Triage agent",
    instructions="Handoff to the appropriate agent based on the language of the request.",
    handoffs=[
            handoff(urdu_agent, on_handoff=lambda ctx: on_handoff(urdu_agent, ctx)),
            handoff(english_agent, on_handoff=lambda ctx: on_handoff(english_agent, ctx))
    ],
)


async def main(input: str):
    result = await Runner.run(triage_agent, input=input, run_config=config)
    print(result.final_output)


In [38]:
asyncio.run(main('tell me about my billing status. traslate it to urdu'))

--------------------------------
Handing off to Urdu agent...
--------------------------------
میں آپ کی بلنگ اسٹیٹس کے بارے میں جاننے میں مدد کر سکتا ہوں۔ براہ کرم مجھے اپنی بلنگ کے بارے میں مزید تفصیلات بتائیں، جیسے کہ آپ کا اکاؤنٹ نمبر، نام، یا بل کی تاریخ۔ اس معلومات کے ساتھ، میں آپ کو آپ کی بلنگ اسٹیٹس کے بارے میں معلومات دے سکتا ہوں۔

(Agar aap apni billing status ke bare mein maloomat chahte hain, to mujhe apne account number, naam, ya bill ki tareekh jaisi tafsilat batayen. Is maloomat ke sath, main aapko aapki billing status ke bare mein maloomat de sakta hoon.)

